<a href="https://colab.research.google.com/github/chuahwb/FNB-Imagery-AI-Tool/blob/main/notebooks/creative_description_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
IPython Notebook for Phase 4: Creative Description Augmentation

This notebook implements the "Creative Director" agent approach.
It takes selected descriptions (in JSONL format) from Phase 2/3, defines
marketing targets (either manually or via LLM), and uses a powerful LLM
to generate creative variations as structured data (Pydantic models)
for augmentation.

V9 Changes:
- Implemented robust JSON extraction logic in `generate_marketing_targets`
  to handle extra text around the JSON output from the LLM.
- (Inherits V8 changes: Increased Strategist Tokens, Optional Target Fields, Retries, Strengthened Prompt, GDrive Input, Detailed Tokens, Gemini Model, JSONL Input, Input Validation, JSON Ref in Prompt, Expanded Options, Structured Output, Params)
"""

'\nIPython Notebook for Phase 4: Creative Description Augmentation\n\nThis notebook implements the "Creative Director" agent approach.\nIt takes selected descriptions (in JSONL format) from Phase 2/3, defines\nmarketing targets (either manually or via LLM), and uses a powerful LLM\nto generate creative variations as structured data (Pydantic models)\nfor augmentation.\n\nV9 Changes:\n- Implemented robust JSON extraction logic in `generate_marketing_targets`\n  to handle extra text around the JSON output from the LLM.\n- (Inherits V8 changes: Increased Strategist Tokens, Optional Target Fields, Retries, Strengthened Prompt, GDrive Input, Detailed Tokens, Gemini Model, JSONL Input, Input Validation, JSON Ref in Prompt, Expanded Options, Structured Output, Params)\n'

In [ ]:
# @title Setup: Install Libraries and Import Modules
# Install necessary libraries
!pip install instructor openai python-dotenv pandas tqdm Jinja2 -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.6/345.6 kB 11.3 MB/s eta 0:00:00


In [ ]:
import os
import base64
import instructor
from openai import OpenAI
# Make sure Pydantic v2+ is used
from pydantic import BaseModel, Field, field_validator, PrivateAttr, ValidationError
from PIL import Image
from io import BytesIO
from typing import List, Dict, Any, Optional, Tuple
import pandas as pd
from tqdm.notebook import tqdm
from dotenv import load_dotenv
import time
import traceback # For detailed error logging
import json # For potentially formatting descriptions for prompt
import html # For escaping text
import sys # For potentially exiting early if input file missing
from google.colab import drive # Import Google Drive library
import datetime

In [ ]:
# @title Mount Google Drive and Set Input File Path
# This cell mounts your Google Drive to access files stored there.
# It will prompt you for authorization the first time you run it.

try:
    drive.mount('/content/drive')
    print("✅ Google Drive mounted successfully.")

    # --- Define Input File Path on Google Drive ---
    # ** IMPORTANT: Update the filename if needed **
    DRIVE_BASE_PATH = '/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset_imagedesc'
    INPUT_FILENAME = 'image_descriptions_2025-04-22_19-22-30.jsonl' # Assuming this is the output from Phase 2/3
    INPUT_DESCRIPTIONS_FILE = os.path.join(DRIVE_BASE_PATH, INPUT_FILENAME)

    # Check if the directory and file exist after mounting
    if not os.path.isdir(DRIVE_BASE_PATH):
        print(f"❌ Error: Google Drive directory not found: {DRIVE_BASE_PATH}")
        print("   Please ensure the path is correct and Drive is mounted properly.")
        INPUT_DESCRIPTIONS_FILE = None # Prevent proceeding if path is wrong
    elif not os.path.exists(INPUT_DESCRIPTIONS_FILE):
         print(f"⚠️ Warning: Input file not found at the specified path: {INPUT_DESCRIPTIONS_FILE}")
         print(f"   Ensure the file '{INPUT_FILENAME}' exists in the directory.")
         # Keep the path, the loading logic later will handle the FileNotFoundError
    else:
        print(f"✅ Input file path set to: {INPUT_DESCRIPTIONS_FILE}")

except Exception as e:
    print(f"❌ An error occurred during Google Drive mounting or path setting: {e}")
    INPUT_DESCRIPTIONS_FILE = None # Prevent proceeding on error

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully.
✅ Input file path set to: /content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset_imagedesc/image_descriptions_2025-04-22_19-22-30.jsonl


In [ ]:

# @title Configure API Key and OpenRouter Client

# --- IMPORTANT ---
# Set your OpenRouter API key (using .env file or environment variable recommended)
dotenv_path = "/content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/colab_secrets/.env"

if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path)
    print(f"Loaded .env file from path: {dotenv_path}")
else:
    print(f"Error: .env file not found at path: {dotenv_path}")

OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY_1")

if not OPENROUTER_API_KEY:
    print("⚠️ OpenRouter API Key not found.")
    # OPENROUTER_API_KEY = input("Enter your OpenRouter API Key: ")
if not GEMINI_API_KEY:
    print("⚠️ Gemini API Key not found.")
    # GEMINI_API_KEY = input("Enter your Gemini API Key: ")

# Select LLM service provider
LLM_SERVICE_PROVIDER = "OpenRouter" # or "Gemini" or "openai" or "OpenRouter"
if LLM_SERVICE_PROVIDER == "OpenRouter":
  CLIENT_BASE_URL = "https://openrouter.ai/api/v1"
  CLIENT_API_KEY = OPENROUTER_API_KEY
elif LLM_SERVICE_PROVIDER == "Gemini":
  CLIENT_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
  CLIENT_API_KEY = GEMINI_API_KEY

# Configure the Instructor client to use OpenRouter
# Patching the client to enable structured responses (Pydantic models)
client = instructor.patch(
    OpenAI(
        base_url=CLIENT_BASE_URL,
        api_key=CLIENT_API_KEY,
        default_headers={
            "HTTP-Referer": "http://localhost:8888", # Replace with your app URL
            "X-Title": "F&B Creative Augmentation", # Replace with your app name
        },
        timeout=900 # Increased timeout further for potentially very long structured responses
    ),
    mode=instructor.Mode.MD_JSON # Use Markdown JSON mode for better compatibility
)

print(f"✅ OpenAI client patched with Instructor and configured for {LLM_SERVICE_PROVIDER}.")

Loaded .env file from path: /content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/colab_secrets/.env
✅ OpenAI client patched with Instructor and configured for OpenRouter.


In [ ]:
# @title Define Pydantic Models for Structured Output

# Re-define the core description model (matching the one used for input)
class FnbImageDescription(BaseModel):
    """Structured description of an F&B social media image."""
    primary_subject: str = Field(..., description="Detailed description of the main food, drink, person, or element.")
    composition_framing: str = Field(..., description="Description of layout, camera angle, and framing.")
    background_environment: str = Field(..., description="Details of the setting, including prominent foreground elements, background elements, surfaces, other objects, and depth of field.")
    lighting_color: str = Field(..., description="Description of light source, style, direction, shadows, highlights, colors, and temperature.")
    texture_materials: str = Field(..., description="Specific textures visible for key materials.")
    text_branding: str = Field(..., description="Description of the style, placement, and purpose of visible text and branding elements.")
    mood_atmosphere: str = Field(..., description="Overall feeling conveyed by the image.")
    overall_style: str = Field(..., description="Characterization of the overall image style including technical details (camera effects, lighting style, rendering).")

    # Optional: Add a validator to ensure fields are not empty
    @field_validator('*', mode='before')
    def check_not_empty(cls, value):
        if isinstance(value, str) and not value.strip():
            return "(Not specified)"
        return value

# Define a model for a single creative variation, including its target and structured description
class CreativeVariation(BaseModel):
    """Represents a single generated creative variation."""
    target_audience: Optional[str] = Field(None, description="The target audience for this variation.")
    target_objective: Optional[str] = Field(None, description="The marketing objective for this variation.")
    target_niche: Optional[str] = Field(None, description="The content niche/theme for this variation.")
    target_voice: Optional[str] = Field(None, description="The brand voice for this variation.") # Made Optional
    generated_description: FnbImageDescription = Field(..., description="The detailed structured description of the new visual concept.")
    creative_reasoning: Optional[str] = Field(None, description="Optional: Brief explanation of the creative choices made for this variation.")

# Define the main response model expected from the Creative Director LLM
class AugmentedVariationsResponse(BaseModel):
    """The overall response containing a list of generated creative variations."""
    variations: List[CreativeVariation] = Field(..., description="A list containing all the generated creative variations.")
    # Add PrivateAttr to potentially store raw response if needed later, though instructor handles parsing
    _raw_response: Optional[Any] = PrivateAttr(default=None)


print("✅ Pydantic models for structured input/output defined.")

✅ Pydantic models for structured input/output defined.


In [ ]:
# @title Define Expanded Marketing Dimensions and Options Pool

# This dictionary holds the potential options the LLM can use if generating targets (Method B)
MARKETING_OPTIONS = {
    "Target Audience": [
        # Demographics
        "Gen Z (18-24)", "Millennials (25-40)", "Gen X (41-56)", "Baby Boomers (57+)",
        "Young Professionals", "Students (Budget-conscious)", "Families with Young Kids", "Families with Teens",
        "Couples", "Singles", "Seniors",
        # Psychographics/Interests
        "Foodies/Gourmands", "Health-Conscious Individuals", "Fitness Enthusiasts", "Luxury Seekers",
        "Casual Diners", "Comfort Food Seekers", "Adventurous Eaters", "Busy Commuters", "Nightlife Crowd",
        "Brunch Enthusiasts", "Coffee Lovers", "Cocktail Aficionados", "Wine Connoisseurs", "Craft Beer Fans",
        "Home Cooks", "Baking Enthusiasts",
        # Location/Context
        "Local Residents", "Tourists", "Office Workers (Lunch)", "Event Attendees (Nearby)", "Remote Workers",
        # Dietary Needs
        "Vegans", "Vegetarians", "Pescatarians", "Flexitarians", "Gluten-Free Seekers", "Dairy-Free",
        "Keto/Low-Carb Followers", "Paleo Followers", "Halal Consumers", "Kosher Consumers", "Allergy Sufferers (Specify)"
    ],
    "Business Objective": [
        # Awareness
        "Increase Brand Visibility", "Build Brand Image (Premium/Fun/Healthy/Sustainable/etc.)", "Introduce New Menu Item",
        "Announce New Location/Chef/Event/Partnership", "Educate Audience (ingredients/process/history)", "Generate Buzz",
        # Consideration
        "Drive Website Traffic", "Encourage Menu Views", "Increase Foot Traffic", "Promote Event Attendance/Sign-ups",
        "Generate Leads (e.g., catering inquiries)", "Build Community", "Increase Engagement (Likes, Shares, Comments, Saves)",
        "Generate Discussion/Questions", "Position as Thought Leader",
        # Conversion
        "Drive Online Orders/Reservations", "Promote Specific Offer/Discount", "Push Limited-Time Offer (LTO)",
        "Increase Average Order Value (e.g., promote add-ons)", "Boost Loyalty Program Sign-ups", "Sell Merchandise",
        # Advocacy
        "Encourage User-Generated Content (UGC)", "Generate Positive Reviews/Testimonials", "Build Brand Trust/Authenticity",
        "Encourage Referrals"
    ],
    "Content Niche/Theme": [
        # Product Focus
        "Ingredient Spotlight", "Appetite Appeal (Close-up/Texture Focus/Action Shot)", "New Dish Feature", "Drink Special/Pairing",
        "Combo Meal / Value Offer", "Seasonal Menu Item", "Holiday Special", "Behind-the-Scenes (Food Prep/Plating)",
        "Food Porn", "Deconstructed Dish", "Menu Item Comparison",
        # Experience Focus
        "Ambiance Highlight (Interior/Exterior/Patio/Decor Detail)", "Live Music/Entertainment Feature", "Customer Experience/Testimonials",
        "Lifestyle Integration (Product in Context/Use Case)", "Staff Profile/Behind-the-Scenes (People/Culture)",
        "Community Event Feature", "User-Generated Content Feature", "Takeout/Delivery Experience", "Pet-Friendly Feature",
        # Information/Story Focus
        "How-To / Recipe Idea", "Food History / Storytelling", "Sustainability/Sourcing Story", "Chef's Story/Philosophy",
        "Health Benefits / Nutrition Info", "Food Pairing Guide", "Cooking Tips", "Local Partnership Spotlight",
        # Time/Occasion Focus
        "Breakfast/Brunch Focus", "Lunch Special", "Happy Hour", "Dinner Feature", "Late Night Menu", "Weekend Special",
        "Celebration/Special Occasion (Birthday, Anniversary)", "Quick Bite / Grab-and-Go", "Catering Offer", "Meal Prep Idea"
    ],
    "Brand Voice": [
        # Tone
        "Playful", "Humorous", "Witty", "Sarcastic", "Sophisticated", "Elegant", "Luxurious", "Minimalist",
        "Rustic", "Cozy", "Homely", "Warm", "Friendly", "Approachable", "Casual", "Conversational",
        "Modern", "Bold", "Energetic", "Vibrant", "Exciting", "Calm", "Relaxed", "Serene", "Authentic", "Honest", "Transparent",
        # Style
        "Informative", "Educational", "Authoritative", "Expert", "Inspirational", "Aspirational", "Motivational",
        "Quirky", "Edgy", "Provocative", "Nostalgic", "Storytelling", "Empathetic", "Supportive", "Community-Focused"
    ]
}

print("✅ Expanded Marketing dimensions and options pool defined.")

✅ Expanded Marketing dimensions and options pool defined.


In [ ]:
# @title Configuration for Augmentation

# --- General Settings ---
now = datetime.datetime.now()
# Use underscores in timestamp for filename safety
formatted_date_time = now.strftime("%Y-%m-%d_%H-%M-%S")

OUTPUT_AUGMENTED_FILE = f"augmented_descriptions_gemini_{formatted_date_time}.csv" # Output file for structured variations
# N: How many variations to generate per input (Structured output is token-heavy, start lower)
NUM_VARIATIONS_PER_REFERENCE = 5 # Start with 3-4 when requesting structured JSON, test increasing later

# --- LLM Selection ---
# Updated to use Gemini 2.5 Pro Experimental Free
'''
CREATIVE_DIRECTOR_MODEL = "google/gemini-2.5-pro-preview-03-25"
TARGET_STRATEGIST_MODEL = "google/gemini-2.5-pro-preview-03-25"
'''
# Free Gemini 2.5 pro
'''
CREATIVE_DIRECTOR_MODEL = "gemini-2.5-pro-exp-03-25"
TARGET_STRATEGIST_MODEL = "gemini-2.5-pro-exp-03-25"
'''


CREATIVE_DIRECTOR_MODEL = "openai/gpt-4.1-mini"
TARGET_STRATEGIST_MODEL = CREATIVE_DIRECTOR_MODEL
OUTPUT_AUGMENTED_FILE = 'augmented_descriptions_'+CREATIVE_DIRECTOR_MODEL.replace('/','_').replace(':','_')+'.csv'


# --- Target Generation Strategy ---
# 'MANUAL': Use the MANUAL_TARGETS list defined below.
# 'LLM_GENERATED': Use an LLM call (generate_marketing_targets function) to create targets.
TARGET_GENERATION_METHOD = 'LLM_GENERATED' # 'MANUAL' or 'LLM_GENERATED'

# --- Manual Targets (Used if TARGET_GENERATION_METHOD = 'MANUAL') ---
# Define N specific, combined targets. Ensure the number matches NUM_VARIATIONS_PER_REFERENCE.
MANUAL_TARGETS = [
    {
        "audience": "Young Professionals (25-35)", "objective": "Increase Brand Awareness",
        "niche": "Ambiance Highlight", "voice": "Sophisticated"
    },
    {
        "audience": "Foodies/Gourmands", "objective": "Highlight Premium Quality",
        "niche": "Ingredient Spotlight", "voice": "Passionate"
    },
    {
        "audience": "Casual Diners", "objective": "Promote Specific Offer/Discount",
        "niche": "Discount/Deal Feature", "voice": "Playful"
    },
    # Add more targets if increasing NUM_VARIATIONS_PER_REFERENCE
]

# Adjust N if manual list length mismatch
if TARGET_GENERATION_METHOD == 'MANUAL' and len(MANUAL_TARGETS) != NUM_VARIATIONS_PER_REFERENCE:
    print(f"⚠️ Warning: NUM_VARIATIONS_PER_REFERENCE is {NUM_VARIATIONS_PER_REFERENCE} but {len(MANUAL_TARGETS)} MANUAL_TARGETS are defined.")
    NUM_VARIATIONS_PER_REFERENCE = len(MANUAL_TARGETS)
    print(f"   Adjusted NUM_VARIATIONS_PER_REFERENCE to {NUM_VARIATIONS_PER_REFERENCE}.")

# --- Model Pricing (Optional - for estimation if using non-free models later) ---
# Add pricing if you switch back to paid models. Free models have $0 cost.
MODEL_PRICING = {
    "google/gemini-2.5-pro-preview-03-25": {"input_cost_per_M": 1.25, "output_cost_per_M": 10.00},
    # Add other models if needed
    # "openai/gpt-4o": {"input_cost_per_M": 5.00, "output_cost_per_M": 15.00},
    # "anthropic/claude-3.7-sonnet": {"input_cost_per_M": 3.00, "output_cost_per_M": 15.00},
    #"openai/gpt-4o": {"input_cost_per_M": 5.00, "output_cost_per_M": 15.00}
}
print("⚠️ MODEL_PRICING set for the free Gemini model. Update if using paid models.")


print(f"--- Configuration Summary ---")
print(f"Input File: {INPUT_DESCRIPTIONS_FILE}")
print(f"Output File: {OUTPUT_AUGMENTED_FILE}")
print(f"Variations per Reference (N): {NUM_VARIATIONS_PER_REFERENCE}")
print(f"Creative Director Model: {CREATIVE_DIRECTOR_MODEL}")
print(f"Target Generation Method: {TARGET_GENERATION_METHOD}")
if TARGET_GENERATION_METHOD == 'LLM_GENERATED':
    print(f"Target Strategist Model: {TARGET_STRATEGIST_MODEL}")
print("---")

⚠️ MODEL_PRICING set for the free Gemini model. Update if using paid models.
--- Configuration Summary ---
Input File: /content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset_imagedesc/image_descriptions_2025-04-22_19-22-30.jsonl
Output File: augmented_descriptions_openai_gpt-4.1-mini.csv
Variations per Reference (N): 5
Creative Director Model: openai/gpt-4.1-mini
Target Generation Method: LLM_GENERATED
Target Strategist Model: openai/gpt-4.1-mini
---


In [ ]:
# @title Helper Function: Format Reference Description for Prompt (Using JSON)

def format_reference_for_prompt(description_row: pd.Series) -> str:
    """Formats the structured description from a DataFrame row into a JSON string for the prompt."""
    fields_to_include = [
        "primary_subject", "composition_framing", "background_environment",
        "lighting_color", "texture_materials", "text_branding",
        "mood_atmosphere", "overall_style"
    ]
    prompt_dict = {}
    if 'Category' in description_row:
        prompt_dict['Category'] = description_row['Category']
    for field in fields_to_include:
        value = description_row.get(field, '(Not specified)')
        value_cleaned = html.unescape(str(value))
        prompt_dict[field] = value_cleaned
    try:
        return json.dumps(prompt_dict, indent=2)
    except Exception as e:
        print(f"⚠️ Error converting reference description row to JSON: {e}")
        return str(prompt_dict)

print("✅ Helper function 'format_reference_for_prompt' confirmed to output JSON string.")

✅ Helper function 'format_reference_for_prompt' confirmed to output JSON string.


In [ ]:
# @title Option B: Function to Generate Marketing Targets using LLM (Robust Parsing)

def extract_json_block(text: str) -> Optional[str]:
    """Extracts the first valid JSON list or object block from text, handling markdown."""
    # Check for markdown code fence first
    if "```json" in text:
        start_marker = text.find("```json") + len("```json")
        end_marker = text.rfind("```")
        if start_marker != -1 and end_marker != -1 and end_marker > start_marker:
            text = text[start_marker:end_marker].strip() # Extract content within fences
        else: # Malformed markdown, try searching whole string
             pass

    # Find the first opening bracket/brace and the last closing bracket/brace
    json_start = text.find('[')
    obj_start = text.find('{')

    # Determine the actual start index (prefer list, fallback to object)
    if json_start != -1 and (obj_start == -1 or json_start < obj_start):
        start_index = json_start
        end_marker_char = ']'
    elif obj_start != -1:
        start_index = obj_start
        end_marker_char = '}'
    else:
        # No JSON start marker found
        return None

    # Find the last corresponding closing marker
    end_index = text.rfind(end_marker_char)

    if start_index != -1 and end_index != -1 and end_index > start_index:
        return text[start_index : end_index + 1]
    else:
        # No valid JSON block found
        return None

def generate_marketing_targets(
    reference_description_str: str,
    category: str,
    options_pool: Dict[str, List[str]],
    num_targets: int,
    model_name: str
) -> Tuple[List[Dict[str, str]], Optional[Dict[str, int]]]: # Return usage info
    """
        Uses an LLM to generate N relevant marketing target combinations with robust JSON parsing.
    Returns a tuple: (list_of_targets, usage_dict)
    """
    targets = []
    usage_info = {"prompt_tokens": 0, "completion_tokens": 0, "total_tokens": 0} # Default usage
    print(f"   Generating {num_targets} marketing targets using {model_name}...")

    options_str = json.dumps(options_pool, indent=2)
    prompt = f"""
You are an expert F&B Marketing Strategist.
Your task is to generate {num_targets} distinct and strategically harmonious marketing target combinations that would be suitable for creating visual content variations based on the following reference image description and category.

Reference Image Category: {category}

Reference Image Description (JSON format):
---
{reference_description_str}
---

Available Marketing Options Pool:
---
{options_str}
---

Instructions:
1. Analyze the reference description (content and style) and category.
2. Select {num_targets} distinct combinations of 'Target Audience', 'Business Objective', 'Content Niche/Theme', and 'Brand Voice' from the options pool.
3. Ensure each combination is strategically sound and relevant to the reference image context.
4. Ensure the dimensions within each combination are harmonious.
5. Output ONLY a valid JSON list containing exactly {num_targets} dictionary objects. Each dictionary MUST have the keys "audience", "objective", "niche", "voice", with string values selected from the options pool.

Example Output Format:
[
  {{
    "audience": "Example Audience 1",
    "objective": "Example Objective 1",
    "niche": "Example Niche 1",
    "voice": "Example Voice 1"
  }},
  // ... up to {num_targets} targets
]

Provide ONLY the JSON list in your response. Do not include any conversational text or reasoning before or after the JSON list.
"""
    try:
        # Use a standard client here as we just need the text/JSON response
        # We need the raw response to potentially get usage info
        raw_client = OpenAI(
             base_url=CLIENT_BASE_URL,
             api_key=CLIENT_API_KEY,
             default_headers={ "HTTP-Referer": "http://localhost:8888", "X-Title": "F&B Target Gen"},
             timeout=600
        )
        response = raw_client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.5,
            max_tokens=2048, # Increased max_tokens
            # Note: response_format={"type": "json_object"} might still be useful
            # but we implement robust parsing anyway. Remove if causing issues.
            # response_format={"type": "json_object"}
        )
        generated_targets_full_str = response.choices[0].message.content

        # --- Extract Usage ---
        if hasattr(response, 'usage') and response.usage:
             raw_usage = response.usage
             print(f"Response: {response}")
             if hasattr(raw_usage, 'prompt_tokens') and hasattr(raw_usage, 'completion_tokens'):
                 usage_info = {
                     "prompt_tokens": raw_usage.prompt_tokens,
                     "completion_tokens": raw_usage.completion_tokens,
                     "total_tokens": raw_usage.total_tokens or (raw_usage.prompt_tokens + raw_usage.completion_tokens),
                 }
                 print(f"      Strategist Usage: {usage_info}")
        else:
             print(f"   ⚠️ Usage info not found in Strategist response.")

        # --- Robust JSON Parsing ---
        extracted_json_str = extract_json_block(generated_targets_full_str)

        if extracted_json_str:
            try:
                targets_list = json.loads(extracted_json_str)
                # Handle potential wrapping key (less likely if parsing extracted block)
                if isinstance(targets_list, dict) and len(targets_list) == 1:
                     potential_list = next(iter(targets_list.values()))
                     if isinstance(potential_list, list):
                         targets_list = potential_list

                if isinstance(targets_list, list) and len(targets_list) >= num_targets and all(isinstance(t, dict) for t in targets_list):
                     targets = targets_list[:num_targets]
                     print(f"   ✅ Successfully generated and parsed {len(targets)} targets from extracted JSON.")
                     for t in targets: # Basic key validation
                         if not all(k in t for k in ["audience", "objective", "niche", "voice"]):
                             print(f"   ⚠️ Warning: Generated target missing keys: {t}")
                     return targets, usage_info # Return targets and usage
                else:
                    print(f"   ❌ Error: Extracted JSON was not a valid list of {num_targets} dictionaries.")
                    print(f"   Extracted JSON: {extracted_json_str}")
                    return [], usage_info # Return empty list but still return usage
            except json.JSONDecodeError as json_err:
                 print(f"   ❌ Error decoding extracted JSON response from target generation LLM: {json_err}")
                 print(f"   Extracted String: {extracted_json_str}")
                 return [], usage_info
        else:
            print(f"   ❌ Error: Could not extract JSON block from LLM response.")
            print(f"   Raw Response: {generated_targets_full_str}")
            return [], usage_info

    except Exception as e:
        print(f"   ❌ Error calling target generation LLM ({model_name}): {e}")
        traceback.print_exc()
        return [], usage_info # Return empty list and default usage on error

print("✅ Optional function 'generate_marketing_targets' updated with robust JSON parsing.")


✅ Optional function 'generate_marketing_targets' updated with robust JSON parsing.


In [ ]:
# @title Define Creative Director Prompt Template (Requesting Structured JSON Output)

# Updated prompt to request structured JSON output matching AugmentedVariationsResponse
# Reference description placeholder now expects a JSON string
CREATIVE_DIRECTOR_PROMPT_TEMPLATE = """
You are an expert Creative Director specializing in F&B social media marketing. You have deep knowledge of graphic design principles (color harmony, composition, visual hierarchy, typography), current social media trends, and what drives engagement (attention-grabbing visuals, clear messaging, emotional connection) for different F&B categories. You are highly imaginative and skilled at translating marketing strategy into compelling visual concepts.

Your task is based on the following reference image description (in JSON format) and category:

Reference Image Category: {category}

Reference Image Description (use this for inspiration):
---
{reference_description}
---

Instructions:
1.  **Identify Core Essence:** First, internally identify the core visual essence or 2-4 key defining elements of the Reference Image Description provided (e.g., 'the specific product itself', 'the interaction between people', 'the unique artistic style', 'the specific ambiance').
2.  **Generate Variations:** Generate {num_variations} distinct creative variations based on the Marketing Targets provided below. Each variation MUST be tailored to its specific Marketing Target. Each variation should describe a **new potential image** inspired by the reference but significantly adapted as a **wholesome new visual concept** optimized for its specific marketing target.

Guidance for Variations:
* Use your creative expertise. Go beyond simple 1-to-1 swaps of elements or colors.
* Variations can involve **significant or even drastic changes** to mood, lighting, composition, color palette (maintaining harmony), adding/removing/changing multiple elements (props, backgrounds, people, food styling), introducing relevant text overlays, or shifting the primary focus entirely.
* **Crucially:** Ensure each variation clearly relates back to or reinterprets the **Core Essence** you identified from the reference description. Don't generate completely unrelated concepts. The connection might be thematic, stylistic, or compositional, but it should be discernible.
* Translate the marketing goals into visual elements (e.g., for 'Drive Traffic for Happy Hour' targeting 'Students', consider energetic visuals, maybe showing drink specials prominently with clear price/time text overlay, using vibrant colors and a fun atmosphere. For 'Highlight Premium Quality' targeting 'Foodies', consider close-ups emphasizing texture, perhaps using dramatic lighting and minimalist plating).
* Ensure each variation's description fields are detailed and comprehensive.

Constraints:
* All variations MUST adhere to strong graphic design principles (visual appeal, harmony, clarity).
* The final described image MUST be effective for social media engagement within the context of its specified Marketing Target and the original F&B category.
* A key constraint is that each variation must feel recognizably **inspired by** the provided reference description, even if the theme, mood, or elements are significantly altered to fit the marketing target.
* Generate exactly {num_variations} distinct variations, one for each target below.

Marketing Targets for Variations:
---
{marketing_targets_formatted}
---

Output Format:
You MUST output a single, valid JSON object. This object must contain ONLY a list called "variations".
Each item in the "variations" list must be a JSON object with the following keys EXACTLY:
1.  "target_audience": The target audience string for this variation (must match input target).
2.  "target_objective": The marketing objective string for this variation (must match input target).
3.  "target_niche": The content niche/theme string for this variation (must match input target).
4.  "target_voice": The brand voice string for this variation (must match input target).
5.  "generated_description": A nested JSON object containing the detailed description of the new visual concept, with keys matching the FnbImageDescription structure: "primary_subject", "composition_framing", "background_environment", "lighting_color", "texture_materials", "text_branding", "mood_atmosphere", "overall_style". All these keys must be present with detailed string values.
6.  "creative_reasoning": (Optional but recommended) A brief string explaining the creative choices made for this variation and how they connect to the marketing target and the identified Core Essence.

**CRITICAL:** Ensure every object in the 'variations' list includes *all* specified keys (target_audience, target_objective, target_niche, target_voice, generated_description, creative_reasoning if provided). Pay special attention to including 'target_voice'. The 'generated_description' object MUST contain all 8 required description keys.

Provide ONLY the valid JSON object in your response. Do not include any text, explanations, or markdown formatting before or after the JSON object. Ensure the JSON is well-formed.
"""

print("✅ Creative Director prompt template updated for structured JSON output, JSON reference input, and Core Essence/Inspired By instructions.")

✅ Creative Director prompt template updated for structured JSON output, JSON reference input, and Core Essence/Inspired By instructions.


In [ ]:
# @title Main Augmentation Workflow (Using Instructor for Structured Output & Detailed Token Tracking)

# --- Load Reference Descriptions ---
reference_df = pd.DataFrame() # Initialize empty DataFrame
load_success = False
# Check if INPUT_DESCRIPTIONS_FILE was set correctly after mounting drive
if 'INPUT_DESCRIPTIONS_FILE' in locals() and INPUT_DESCRIPTIONS_FILE and os.path.exists(INPUT_DESCRIPTIONS_FILE):
  try:
          reference_df = pd.read_json(INPUT_DESCRIPTIONS_FILE, lines=True, orient='records')
          if reference_df.empty:
              print(f"⚠️ Warning: Input file {INPUT_DESCRIPTIONS_FILE} is empty.")
          else:
              print(f"✅ Loaded {len(reference_df)} reference descriptions from {INPUT_DESCRIPTIONS_FILE}")
              load_success = True # Flag success
  except Exception as e:
      print(f"❌ Error loading input file {INPUT_DESCRIPTIONS_FILE}: {e}")
    # Keep load_success as False
elif 'INPUT_DESCRIPTIONS_FILE' in locals() and INPUT_DESCRIPTIONS_FILE:
    print(f"❌ Error: Input file path set, but file not found at {INPUT_DESCRIPTIONS_FILE}. Please check the path and filename in the 'Mount Google Drive' cell.")
else:
    print("❌ Error: Input file path was not set correctly (INPUT_DESCRIPTIONS_FILE variable is missing or None). Please run the 'Mount Google Drive' cell.")

# --- Augmentation Loop ---
augmented_results_list = [] # Store results as flat dictionaries
if load_success and not reference_df.empty: # Proceed only if loading was successful and df is not empty
    for index, row in tqdm(reference_df.iterrows(), total=len(reference_df), desc="Augmenting Descriptions"):
        image_id = row.get('Image ID', f'UnknownID_{index}')
        category = row.get('Category', 'UnknownCategory')
        # Format reference description as JSON string
        reference_desc_json_str = format_reference_for_prompt(row)
        print(f"\nAugmenting for Image ID: {image_id} (Category: {category})")

        # --- Initialize token/cost counters for this reference image ---
        strategist_prompt_tokens = 0
        strategist_completion_tokens = 0
        strategist_total_tokens = 0
        cd_prompt_tokens = 0
        cd_completion_tokens = 0
        cd_total_tokens = 0
        estimated_cost = 0.0 # Combined cost for this reference image

        # 1. Define/Generate Targets for this reference image
        current_targets = []
        strategist_usage = None
        if TARGET_GENERATION_METHOD == 'MANUAL':
            current_targets = MANUAL_TARGETS
            print(f"   Using {len(current_targets)} manually defined targets.")
        elif TARGET_GENERATION_METHOD == 'LLM_GENERATED':
            current_targets, strategist_usage = generate_marketing_targets( # Capture usage
                reference_description_str=reference_desc_json_str,
                category=category,
                options_pool=MARKETING_OPTIONS,
                num_targets=NUM_VARIATIONS_PER_REFERENCE,
                model_name=TARGET_STRATEGIST_MODEL
            )
            if strategist_usage:
                strategist_prompt_tokens = strategist_usage.get("prompt_tokens", 0)
                strategist_completion_tokens = strategist_usage.get("completion_tokens", 0)
                strategist_total_tokens = strategist_usage.get("total_tokens", 0)
                # Calculate strategist cost
                if TARGET_STRATEGIST_MODEL in MODEL_PRICING:
                    pricing = MODEL_PRICING[TARGET_STRATEGIST_MODEL]
                    strat_in_cost = (strategist_prompt_tokens / 1_000_000) * pricing["input_cost_per_M"]
                    strat_out_cost = (strategist_completion_tokens / 1_000_000) * pricing["output_cost_per_M"]
                    estimated_cost += (strat_in_cost + strat_out_cost)
                else:
                     print(f"   ⚠️ Pricing not found for Strategist model {TARGET_STRATEGIST_MODEL}.")

            if not current_targets:
                print(f"   Skipping augmentation for {image_id} due to target generation error.")
                continue # Skip to next reference image

        if not current_targets:
             print(f"   No targets defined for {image_id}. Skipping.")
             continue

        num_actual_targets = len(current_targets) # Use the actual number of targets obtained

        # Format targets for the prompt
        marketing_targets_formatted = ""
        for i, target in enumerate(current_targets):
            marketing_targets_formatted += f"Target {i+1}: Audience={target.get('audience','N/A')}, Objective={target.get('objective','N/A')}, Niche={target.get('niche','N/A')}, Voice={target.get('voice','N/A')}\n"

        print(f"Marketing targets:\n{marketing_targets_formatted}\n")
        time.sleep(11)

        # 2. Construct the Creative Director Prompt
        creative_prompt = CREATIVE_DIRECTOR_PROMPT_TEMPLATE.format(
            category=category,
            reference_description=reference_desc_json_str,
            num_variations=num_actual_targets, # Use actual number
            marketing_targets_formatted=marketing_targets_formatted.strip()
        )

        # 3. Call the Creative Director LLM using Instructor for structured output
        print(f"   Calling Creative Director ({CREATIVE_DIRECTOR_MODEL}) for structured output...")
        structured_response = None
        cd_usage = None
        try:
            start_time = time.time()
            structured_response : AugmentedVariationsResponse = client.chat.completions.create(
                model=CREATIVE_DIRECTOR_MODEL,
                response_model=AugmentedVariationsResponse,
                messages=[{"role": "user", "content": creative_prompt}],
                temperature=0.7,
                max_tokens=8000,
                max_retries=2, # Increased retries to allow self-correction for validation errors
            )
            end_time = time.time()
            print(f"   ✅ Creative Director response received and parsed in {end_time - start_time:.2f} seconds.")

            # Extract overall usage for the CD call
            raw_cd_response = getattr(structured_response, '_raw_response', None)
            if raw_cd_response and hasattr(raw_cd_response, 'usage'):
                 raw_usage = raw_cd_response.usage
                 if raw_usage and hasattr(raw_usage, 'prompt_tokens') and hasattr(raw_usage, 'completion_tokens'):
                     cd_prompt_tokens = raw_usage.prompt_tokens
                     cd_completion_tokens = raw_usage.completion_tokens
                     cd_total_tokens = raw_usage.total_tokens or (cd_prompt_tokens + cd_completion_tokens)
                     cd_usage = { # Store for potential later use/logging
                         "prompt_tokens": cd_prompt_tokens,
                         "completion_tokens": cd_completion_tokens,
                         "total_tokens": cd_total_tokens,
                     }
                     print(f"      Creative Director Usage (Overall): {cd_usage}")
                     # Add CD cost to total estimated cost
                     if CREATIVE_DIRECTOR_MODEL in MODEL_PRICING:
                         pricing = MODEL_PRICING[CREATIVE_DIRECTOR_MODEL]
                         cd_in_cost = (cd_prompt_tokens / 1_000_000) * pricing["input_cost_per_M"]
                         cd_out_cost = (cd_completion_tokens / 1_000_000) * pricing["output_cost_per_M"]
                         estimated_cost += (cd_in_cost + cd_out_cost)
                     else:
                          print(f"   ⚠️ Pricing not found for Creative Director model {CREATIVE_DIRECTOR_MODEL}.")
            else:
                 print(f"   ⚠️ Usage info not found in Creative Director response.")


            # 4. Process and Store Structured Variations
            if structured_response and structured_response.variations:
                num_received_variations = len(structured_response.variations)
                if num_received_variations >= num_actual_targets:
                    variations_to_process = structured_response.variations[:num_actual_targets]
                    if num_received_variations > num_actual_targets:
                         print(f"   ⚠️ Warning: Received {num_received_variations} variations but processing only the first {num_actual_targets} requested.")

                    # Calculate estimated completion tokens per variation
                    est_completion_tokens_per_var = int(cd_completion_tokens / num_actual_targets) if num_actual_targets > 0 else 0

                    for i, variation_obj in enumerate(variations_to_process):
                        # target_info = current_targets[i] # Target info is now directly in variation_obj
                        desc_obj = variation_obj.generated_description

                        # Flatten the result for DataFrame/CSV storage
                        flat_result = {
                            "Reference Image ID": image_id,
                            "Reference Category": category,
                            # Use .get() when accessing optional fields from Pydantic model
                            "Target Audience": variation_obj.target_audience or "N/A",
                            "Target Objective": variation_obj.target_objective or "N/A",
                            "Target Niche": variation_obj.target_niche or "N/A",
                            "Target Voice": variation_obj.target_voice or "N/A", # Handle potential None
                            "Creative Reasoning": variation_obj.creative_reasoning or "", # Handle potential None
                            # Strategist Tokens (same for all variations from this ref)
                            "Strategist Prompt Tokens": strategist_prompt_tokens,
                            "Strategist Completion Tokens": strategist_completion_tokens,
                            "Strategist Total Tokens": strategist_total_tokens,
                            # Creative Director Tokens (overall call, same for all variations from this ref)
                            "CD Prompt Tokens": cd_prompt_tokens,
                            "CD Completion Tokens": cd_completion_tokens,
                            "CD Total Tokens": cd_total_tokens,
                            # Estimated Per Variation Tokens
                            "Est Variation Completion Tokens": est_completion_tokens_per_var,
                            # Total Estimated Cost (Strategist + CD call, same for all variations from this ref)
                            "Total Estimated Cost (USD)": round(estimated_cost, 8), # Increased precision
                            # Add fields from the nested description object, prefixing them
                            "variation_primary_subject": desc_obj.primary_subject,
                            "variation_composition_framing": desc_obj.composition_framing,
                            "variation_background_environment": desc_obj.background_environment,
                            "variation_lighting_color": desc_obj.lighting_color,
                            "variation_texture_materials": desc_obj.texture_materials,
                            "variation_text_branding": desc_obj.text_branding,
                            "variation_mood_atmosphere": desc_obj.mood_atmosphere,
                            "variation_overall_style": desc_obj.overall_style,
                            "Source Model": row.get('Model') # Original model that created reference
                        }
                        augmented_results_list.append(flat_result)
                else:
                    print(f"   ⚠️ Warning: Requested {num_actual_targets} variations but received only {num_received_variations} structured variations for {image_id}.")
                    # Log error or handle partial results if needed
            else:
                 print(f"   ❌ Error: Failed to parse structured variations response or variations list is empty for {image_id}.")

        except ValidationError as val_err:
             print(f"   ❌ Pydantic Validation Error parsing response for {image_id}: {val_err}")
        except Exception as e:
            print(f"   ❌ Error calling/parsing Creative Director LLM for {image_id}: {e}")
            traceback.print_exc()

else:
     print("--- Augmentation workflow skipped because no valid input data was loaded. ---")


print("\n✅ Augmentation workflow finished.")


✅ Loaded 1 reference descriptions from /content/drive/MyDrive/AI Imagery Marketing Tool/Colab Notebook/dataset_imagedesc/image_descriptions_2025-04-22_19-22-30.jsonl


Augmenting Descriptions:   0%|          | 0/1 [00:00<?, ?it/s]


Augmenting for Image ID: menu_0003 (Category: Menu Displays)
   Generating 5 marketing targets using openai/gpt-4.1-mini...
Response: ChatCompletion(id='gen-1745450571-CT9iWZUjFhnbtaqeZJE4', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='[\n  {\n    "audience": "Foodies/Gourmands",\n    "objective": "Introduce New Menu Item",\n    "niche": "New Dish Feature",\n    "voice": "Playful"\n  },\n  {\n    "audience": "Students (Budget-conscious)",\n    "objective": "Promote Specific Offer/Discount",\n    "niche": "Combo Meal / Value Offer",\n    "voice": "Casual"\n  },\n  {\n    "audience": "Young Professionals",\n    "objective": "Drive Online Orders/Reservations",\n    "niche": "Quick Bite / Grab-and-Go",\n    "voice": "Modern"\n  },\n  {\n    "audience": "Millennials (25-40)",\n    "objective": "Increase Engagement (Likes, Shares, Comments, Saves)",\n    "niche": "Appetite Appeal (Close-up/Texture Focus/Action Shot)",\n    "voice": "En

In [ ]:
# @title Save Augmented Descriptions

if augmented_results_list:
    augmented_df = pd.DataFrame(augmented_results_list)
    print(f"\nGenerated {len(augmented_df)} augmented descriptions.")

    def create_variation_prompt(row):
        """Creates a formatted string prompt from variation description fields."""
        prompt_parts = []
        # Define variation column names and their desired labels in the prompt
        # Using 'Reference Category' for context
        fields_to_include = {
            "Reference Category": "Category",
            "variation_primary_subject": "Primary subject",
            "variation_composition_framing": "Composition framing",
            "variation_background_environment": "Background environment",
            "variation_lighting_color": "Lighting color",
            "variation_texture_materials": "Texture materials",
            "variation_text_branding": "Text branding",
            "variation_mood_atmosphere": "Mood atmosphere",
            "variation_overall_style": "Overall style"
            # Add other relevant fields if desired, e.g., Target Audience?
        }

        for col_name, label in fields_to_include.items():
            value = row.get(col_name) # Use .get() for safety
            # Add part if value exists and is not just whitespace
            if pd.notna(value) and str(value).strip():
                prompt_parts.append(f"{label}: {str(value).strip()}")

        return ", ".join(prompt_parts)

    # Apply function AFTER creating DataFrame from list, BEFORE ordering columns
    if not augmented_df.empty:
         augmented_df['image_prompt'] = augmented_df.apply(create_variation_prompt, axis=1)
    else:
        # Add empty column if df happens to be empty initially, prevents error later
         augmented_df['image_prompt'] = pd.Series(dtype='object')

    # Define column order including new token/cost columns
    final_column_order = [
        "Reference Image ID", "Reference Category",
        "Target Audience", "Target Objective", "Target Niche", "Target Voice",
        "Creative Reasoning",
        "Strategist Prompt Tokens", "Strategist Completion Tokens", "Strategist Total Tokens",
        "CD Prompt Tokens", "CD Completion Tokens", "CD Total Tokens",
        "Est Variation Completion Tokens", "Total Estimated Cost (USD)",
        "variation_primary_subject", "variation_composition_framing", "variation_background_environment",
        "variation_lighting_color", "variation_texture_materials", "variation_text_branding",
        "variation_mood_atmosphere", "variation_overall_style",
        "image_prompt", "Source Model"
    ]
    # Ensure all columns are present, fill missing with None or appropriate default
    for col in final_column_order:
        if col not in augmented_df.columns:
            augmented_df[col] = None
    augmented_df = augmented_df[final_column_order] # Enforce order


    # Set display options for the output DataFrame
    pd.set_option('display.max_rows', 50)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 1500) # Wider display
    pd.set_option('display.max_colwidth', 100)
    display(augmented_df.head())

    # --- Optional: Calculate and Display Summary Statistics ---
    print("\n--- Summary Statistics (per Augmentation Call) ---")
    # Group by reference image to get per-call stats, then average
    # Ensure necessary columns exist and handle potential NaN values if errors occurred
    stats_cols = ["Reference Image ID", "Strategist Total Tokens", "CD Total Tokens", "Total Estimated Cost (USD)"]
    if all(col in augmented_df.columns for col in stats_cols):
        call_stats = augmented_df.drop_duplicates(subset=["Reference Image ID"])[stats_cols].copy()
        call_stats.dropna(subset=["Strategist Total Tokens", "CD Total Tokens", "Total Estimated Cost (USD)"], inplace=True) # Drop rows where stats might be missing due to errors

        if not call_stats.empty:
            # Calculate means safely, checking for count > 0
            avg_strat_tokens = call_stats['Strategist Total Tokens'].mean() if call_stats['Strategist Total Tokens'].count() > 0 else 0
            avg_cd_tokens = call_stats['CD Total Tokens'].mean() if call_stats['CD Total Tokens'].count() > 0 else 0
            avg_cost = call_stats['Total Estimated Cost (USD)'].mean() if call_stats['Total Estimated Cost (USD)'].count() > 0 else 0
            total_cost = call_stats['Total Estimated Cost (USD)'].sum()

            print(f"Average Strategist Tokens per Call: {avg_strat_tokens:.0f}")
            print(f"Average Creative Director Tokens per Call (for {NUM_VARIATIONS_PER_REFERENCE} variations): {avg_cd_tokens:.0f}")
            print(f"Average Estimated Cost per Reference Image: ${avg_cost:.6f}")
            print(f"Total Estimated Cost for all runs: ${total_cost:.6f}")
        else:
            print("No successful calls with complete stats to calculate averages from.")
    else:
        print("One or more statistics columns missing from results, cannot calculate summary.")


    # Save to CSV
    try:
        augmented_df.to_csv(OUTPUT_AUGMENTED_FILE, index=False)
        print(f"✅ Augmented descriptions saved to {OUTPUT_AUGMENTED_FILE}")
    except Exception as e:
        print(f"❌ Error saving augmented descriptions to CSV: {e}")
else:
    print("\nNo augmented descriptions were generated.")


Generated 5 augmented descriptions.


,Reference Image ID,Reference Category,Target Audience,Target Objective,Target Niche,Target Voice,Creative Reasoning,Strategist Prompt Tokens,Strategist Completion Tokens,Strategist Total Tokens,CD Prompt Tokens,CD Completion Tokens,CD Total Tokens,Est Variation Completion Tokens,Total Estimated Cost (USD),variation_primary_subject,variation_composition_framing,variation_background_environment,variation_lighting_color,variation_texture_materials,variation_text_branding,variation_mood_atmosphere,variation_overall_style,image_prompt,Source Model
0,menu_0003,Menu Displays,Foodies/Gourmands,Introduce New Menu Item,New Dish Feature,Playful,This variation captures the core essence of the original torn-paper sandwich reveal and handcraf...,1914,239,2153,2735,2405,5140,481,0.0,A vibrant new sandwich creation featuring a glossy brioche bun bursting open to reveal layers of...,"Close-up horizontal shot with the sandwich placed diagonally across the frame, centered with a s...",A textured dark slate serving board with scattered fresh herbs and edible flowers; blurred hints...,"Warm, directional spotlight from the top-left creating dramatic highlights on the glossy bun and...","Glossy, pillowy brioche bun; tender, flaky duck meat; smooth, creamy melted gruyère; juicy cherr...","Stylized playful hand-drawn lettering overlay in bright coral and gold reading ""New! Duck Confit...",Inviting and whimsical with a sense of discovery and indulgence; playful energy combined with go...,High-end food photography with artistic styling; shallow depth of field focusing on sandwich lay...,"Category: Menu Displays, Primary subject: A vibrant new sandwich creation featuring a glossy bri...",openai/o4-mini
1,menu_0003,Menu Displays,Students (Budget-conscious),Promote Specific Offer/Discount,Combo Meal / Value Offer,Casual,"This variation reinterprets the torn-paper sandwich reveal into a straightforward, budget-friend...",1914,239,2153,2735,2405,5140,481,0.0,"A casual, hearty combo meal featuring a stacked sandwich with melted cheddar and grilled chicken...",Top-down flat lay shot capturing the entire combo plate centered on a brightly colored table; sa...,A vibrant cafeteria-style table with bold primary colors (red and yellow) reminiscent of youthfu...,"Bright, even daylight mimicking natural window light; cool white balance to enhance freshness an...","Crunchy, golden fries; soft sandwich bread with melted cheese visibly gooey; cold soda condensat...","Bold, playful typography overlay in white and blue reading ""Student Combo Deal!"" with a large re...","Energetic and fun, with a laid-back vibe that resonates with students looking for affordable, sa...",Bright and colorful food photography with graphic text overlays; flat lay style popular on socia...,"Category: Menu Displays, Primary subject: A casual, hearty combo meal featuring a stacked sandwi...",openai/o4-mini
2,menu_0003,Menu Displays,Young Professionals,Drive Online Orders/Reservations,Quick Bite / Grab-and-Go,Modern,"This concept shifts the playful torn-paper reveal into a polished, modern grab-and-go visual tha...",1914,239,2153,2735,2405,5140,481,0.0,"A sleek, modern takeout sandwich wrapped halfway in branded branded kraft paper, showcasing laye...",Tight medium close-up with vertical framing; sandwich positioned slightly off-center to the righ...,Minimalistic urban café interior with clean lines and muted tones; subtle hints of glass windows...,"Cool, natural daylight with soft shadows; slight directional light from the left to create subtl...","Smooth, thinly sliced turkey; creamy avocado texture; soft Swiss cheese slices; crisp lettuce le...","Minimalist sans-serif typography on kraft paper wrapping reading ""Grab & Go"" with a simple logo ...","Effortless, efficient, and contemporary; conveys convenience without sacrificing quality; design...","Clean, modern food photography with shallow depth of field; natural color grading emphasizing fr...","Category: Menu Displays, Primary subject: A


--- Summary Statistics (per Augmentation Call) ---
Average Strategist Tokens per Call: 2153
Average Creative Director Tokens per Call (for 5 variations): 5140
Average Estimated Cost per Reference Image: $0.000000
Total Estimated Cost for all runs: $0.000000
✅ Augmented descriptions saved to augmented_descriptions_openai_gpt-4.1-mini.csv
